In [4]:
import pandas as pd
from elasticsearch import Elasticsearch
from utils.index_helper import IndexHelper

In [5]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

In [6]:
books_df = pd.read_csv('../data/preprocessed/pp_books.csv')
ratings_df = pd.read_csv('../data/initial/ratings.csv')

ih = IndexHelper(es)

ih.bulk_index(data=books_df, index='books')
ih.bulk_index(data=ratings_df, index='ratings')

C:\Users\georg\Programming\DataSpell\information-retrieval-project\utils\index_helper.py:21: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  if self.es.indices.exists(index=index):
C:\Users\georg\Programming\DataSpell\information-retrieval-project\utils\index_helper.py:22: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  self.es.indices.delete(index=index)


Deleting index...


C:\Users\georg\Programming\DataSpell\information-retrieval-project\utils\index_helper.py:25: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(self.es, _index_data(data_dict, index))
C:\Users\georg\Programming\DataSpell\information-retrieval-project\utils\index_helper.py:21: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  if self.es.indices.exists(index=index):
C:\Users\georg\Programming\DataSpell\information-retrieval-project\utils\index_helper.py:22: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication

Deleting index...


C:\Users\georg\Programming\DataSpell\information-retrieval-project\utils\index_helper.py:25: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(self.es, _index_data(data_dict, index))


In [7]:
def search_books(size: int, query: str, uid: int, percent: float = 0.1):
    fields = ["book_author", "book_title", "summary", "category", "publisher"]
    if query.isnumeric():
        fields.append("year_of_publication")
        
    books_res = es.search(
        index='books',
        size=size,
        query={
            "multi_match": {
                "query": query,
                "type": "most_fields",
                "fields": fields
            }
        },
        scroll='2m'
    )
    
    while len(books_res['hits']['hits']):
        scroll_id = books_res['_scroll_id']
        books_res = es.scroll(scroll_id=scroll_id, scroll='2m')
        if len(books_res['hits']['hits']) == 0:
            break
    
    book_hits = books_res['hits']['hits']
    isbn_list = [book_hit['_source']['isbn'] for book_hit in book_hits]

    ratings_res = es.search(
        index='ratings',
        size=size,
        query={
            "bool": {
                "must": [
                    {"term": {"uid": uid}},
                    {"terms": {"isbn": isbn_list}}
                ]
            }
        })
    rating_hits = ratings_res['hits']['hits']
    rated_isbn_dict = {
        rating_hit['_source']['isbn']: rating_hit['_source']['rating']
        for rating_hit in rating_hits
    }

    for book_hit in book_hits:
        isbn = book_hit['_source']['isbn']
        if isbn in rated_isbn_dict:
            book_hit['_score'] += 2 * rated_isbn_dict[isbn]

    book_hits.sort(key=lambda x: x['_score'], reverse=True)

    book_hits = book_hits[:int(len(book_hits) * percent)]

    return book_hits

In [10]:
term = input("Enter search term: ")
user_id = input("Enter user ID: ")

results = search_books(size=50, query=term, uid=user_id)

C:\Users\georg\AppData\Local\Temp\ipykernel_3388\3729612359.py:6: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  books_res = es.search(
C:\Users\georg\AppData\Local\Temp\ipykernel_3388\3729612359.py:19: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  ratings_res = es.search(


In [11]:
df_columns = ['score', 'isbn', 'book_title', 'book_author', 'summary', 'category', 'publisher', 'year_of_publication']
df_data = []
for item in results:
    df_data.append([
        round(item['_score'], 2),
        item['_source']['isbn'],
        item['_source']['book_title'],
        item['_source']['book_author'],
        item['_source']['summary'],
        item['_source']['category'],
        item['_source']['publisher'],
        item['_source']['year_of_publication']
    ])

pd.DataFrame(columns=df_columns, data=df_data)

,score,isbn,book_title,book_author,summary,category,publisher,year_of_publication
0,23.70,1579541364,"Pets, Part of Family: The Total Care Guide for...",Gary Burghoff,The companion volume to the new Public Televis...,pets,Rodale Press,1999
1,22.13,1889540994,The Wild Life of Pets: A Rubes Cartoon Book (R...,Leigh Rubin,Ever wonder about the secret lives of pets? Or...,pets,BowTie Press,2003
2,21.91,157223217X,Just Labs (Just Pets),Dale C. Spartas,We took our lead titles from the tremendously ...,pets,Willow Creek Press,1999
3,21.34,0716616327,All About Pets (Inspector Mcq),Kathleen Kain,Special mouse detective McQ leads readers on a...,pets,World Book Inc,1995
4,18.41,0816737959,How to Draw Pets,Linda Murray,Provides step-by-step instructions for drawing...,animals in art,Watermill Pr,1995
